In [209]:
# pip install requests
import requests
from bs4 import BeautifulSoup


url = "https://mykbostats.com/players/1"
apikey = "b327144420a12f944bbdd4863c9fc0bffb28a1e3" 
params = {
    "url": url,
    "apikey": apikey,
    "js_render": "true",
    "premium_proxy": "true",
}
response = requests.get("https://api.zenrows.com/v1/", params=params)

#print(response.content[:500])

soup = BeautifulSoup(response.content, 'html.parser')

In [210]:
temp = soup.select('title')[0].text

player = [temp[:temp.find(' - ')], temp[temp.find(' - ')+3:temp.find(' | ')]]

print(player)

['Kang Dong-yeon KBO League Pitching Stats', 'NC Dinos']


In [211]:
# extract table header information
header = soup.select('thead th')

for i in range(len(header)):
  header[i] = header[i].text.strip()

# remove 'Game Stats' data
header = header[:header.index("Date")]

print(header)

['Year', 'Team', 'ERA', 'WHIP', 'W', 'L', 'SV', 'H', 'BSV', 'G', 'GS', 'CG(SHO)', 'QS', 'TBF', 'NP', 'IP', 'R', 'ER', 'H', '2B', '3B', 'HR', 'SO', 'BB', 'IBB', 'HB', 'WP', 'BK']


In [212]:
rows = soup.select('tbody tr')

r = rows[0]

temp = [r.select_one('.left').text.strip(), r.select_one('nobr').text.strip()]
for i in r.select('td')[2:]:
  temp.append(i.text.strip())

temp[1] = player[1]

print(temp)

['2022', 'NC Dinos', '10.38', '2.19', '0', '0', '0', '0', '0', '6', '0', '0 (0)', '0', '44', '157', '8\xa0⅔', '10', '10', '16', '5', '1', '2', '3', '3', '0', '1', '1', '0']


In [ ]:
data = {player[0]:None} # temp[0] is Year info

print(player[0])
print(temp[0])
print(header[1])
print(temp[1])
print(data)


dt = {temp[0]:{}}
for d in range(1, len(header)):
  dt[temp[0]][header[d]] = temp[d]


'''
for d in range(1, len(header)):
  data[player[0]] = {
    temp[0] : {
        header[d] : temp[d]
      }
    }
'''
data[player[0]] = dt
print(data)

SyntaxError: invalid syntax (2054500104.py, line 12)